# Goals of this Clinic

The scope of both calibration and [Dakota](https://dakota.sandia.gov) are large. This two-hour clinic was designed to deliver the following information. 
- Conceptual introduction to calibration and a discussion of some of the major options in calibration methods. 
- Introduction to the [Dakota](https://dakota.sandia.gov) package. 
- Background for a simple example using data from [Clow (2014)]() and a simple model of 1D heat diffusion. 
- Experience with a simple example in Dakota --- including understanding how you must set up your model so Dakota can run it, and what files Dakota uses and makes. 
- Knowledge of what User Guides and resourses are available to further your knowledge.


# Step 1: Introduction to calibration

## What is calibration/optimization/parameter estimation?

- General definition

- black box model (parameters > model > outputs)
- parameters and outputs must be formally defined.

## Some useful definitions

- gradient based vs global
- single vs multi-objective
- complex model vs statistical surogate
- a method that provides parameter uncertainty or just best parameter set (linear vs. nonlinear assumptions)
- constrained vs unconstrained
- smoothness of objetive function?

# Step 2: Problem statement and dataset

Today's clinic will use a model of 1D diffusion of heat in the Earth's crust and data from [Clow (2014)](https://www.earth-syst-sci-data.net/6/201/2014/essd-6-201-2014.pdf).


In [ ]:
import os
import glob
import subprocess
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype

from plotnine import *

In [ ]:
files = glob.glob(os.path.join("resources", "clow_2014", "G10015", "AWU*.txt"))
dfs = []
for path in files:
    date = os.path.split(path)[-1].split(".")[0].split("_")[-1]
    site = os.path.split(path)[-1].split(".")[0].split("_")[0]
    year = int(date[:2])
    if year<19:
        year += 2000
    else:
        year += 1900
    tdf = pd.read_csv(path, header=22, skip_blank_lines=False, sep="\s+")
    tdf["Date"] = date
    tdf["Year"] = year
    tdf["Site"] = site
    dfs.append(tdf)
df = pd.concat(dfs).sort_values(["Site", "Date", "Depth"]).reset_index(drop=True)

(ggplot(df, aes(x="Temperature", y="Depth", color="factor(Year)")) +
 geom_path(na_rm=True) + 
 facet_wrap("~Site") +
 scale_y_reverse())


# Step 3: Model, parameters and objective function.

- diffusion of heat + surface temperature history
- OF based on fitting Clow paper data using an RMSE

# Step 4: Introduction to Dakota

[Dakota](https://dakota.sandia.gov)

[Online Documentation](https://dakota.sandia.gov/content/69-reference-manual)

[PDFs to download](https://dakota.sandia.gov/content/manuals)

- Dakota has more bells and whistles, it is well thought out, and the
  documentation is quite good. Its just extensive and not an iPhone.
- Dakota has a gui if your into that.
- There are lots of hierarchical type things you can do.
- Restart utility, deprepro, and pyprepro....
- Core activity (assuming you have a black box model set up) is to create and
  run an input file.
- Dakota then iteratively runs your model for you to determine parameter values.  

What we will do:
- Look at .in file.
    * discuss each part
- Look at template file and driver.py (connect this with black box parts)
- Run Dakota, create plots, look at output.
- Discuss Dakota's file structure

In [ ]:
! cat analysis/dakota_01_grid.in

In [ ]:
! cat analysis/template_dir/driver.py

In [ ]:
! cat analysis/template_dir/input_template.yaml

In [ ]:
! dakota

In [ ]:
! cat start_01_grid.sh

In [ ]:
os.chdir("analysis")
subprocess.call("./start_01_grid.sh")

In [ ]:
! ls


In [ ]:
os.chdir("MULTIDIM_PARAM")
! ls

In [ ]:
os.chdir("run.1")
! ls

In [ ]:
! cat params.in

In [ ]:
! cat inputs.yml

In [ ]:
! cat results.out

Lets go look at the file strucure. 

look at output make point about reproduciblity and .rst file. 

In [ ]:
! cat dakota_01_grid.out

# Other methods
We just did a brute force grid search. This is sort of an optimization.
Next we will do a gradient based method and a global method.

What's different about their input files. 
- Gradients  are necessary for NL2SOL
- Seed is necessary for EGO (or a default is used)


In [ ]:
os.chdir('..')
os.chdir('..')
! cat dakota_02_nl2sol.in

In [ ]:
! cat dakota_03_ego.in


In [ ]:

subprocess.call("./start_02_nl2sol.sh")
subprocess.call("./start_03_ego.sh")

In [ ]:
files = glob.glob("*.dat")
dfs = []
for file in files:
    df = pd.read_csv(file, engine="python", sep="\s+")
    df["method"] = file.split('.')[0].split("_")[-1]
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)
method_cats = CategoricalDtype(categories=["grid", 
                                           "nl2sol", 
                                           "ego"], 
                               ordered=True)
df["method"] = df["method"].astype(method_cats)
df = df.set_index(["method", "T", "duration_years"]).drop(columns=["interface"])

# plot evaluations
(ggplot(df.reset_index(), aes(x="T", y="duration_years", color="%eval_id")) + 
     geom_point() + 
     scale_color_cmap(name='jet') +
     facet_wrap("~method"))



In [ ]:

(ggplot(df.reset_index(), aes(x="T", y="duration_years", color="rmse")) + 
     geom_point() + 
     facet_wrap("~method"))


In [ ]:
# see how results and number of evaluations are influenced by method
sum_df = df.drop(columns=["%eval_id"]).groupby("method").agg([np.count_nonzero, np.min])
sum_df.columns = sum_df.columns.map('|'.join).str.strip('|')

(ggplot(sum_df.reset_index(), (aes(x="rmse|count_nonzero", y="rmse|amin", color="method"))) + geom_point())


In [ ]:

# summarized best Ts and durations
best_df=df[df.rmse.isin(sum_df["rmse|amin"].values)].reset_index()
print(best_df)

# Discussion:
- computational cost of Dakota method vs complex model evaluation.
    * calculation of numerical gradients
    * increasing dimension
- do you need parameter estimates, or just a best fit point.
- RST file, .out file and reproducible research
- We haven't yet talked about the uncertainty estimates on  parameters, just
  which parameter is best. That is for another day.

# Exploration if time:
* Explore other optimization methods. Start by going to the [Online Reference Manual](https://dakota.sandia.gov/content/69-reference-manual) and selecting Topics Area > Methods > Optimization and Calibration
* Add a second component of the objective function.
* Make the model (of surface temperature history) more complex.
